<a href="https://colab.research.google.com/github/khorneflakes-dev/Proyecto-Final-YELP/blob/main/ETL_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#descomprime el zip con los datasets de yelp
from zipfile import ZipFile 
with ZipFile('drive/MyDrive/Copia de Dataset Yelp.zip', 'r') as zipObj:
  zipObj.extractall('drive/MyDrive/yelp')

In [2]:
import pandas as pd
pd.options.display.max_columns = None # para mostrar todas las columnas de los dataframes
import numpy as np


## obtaining integer user codes ¡suprimible!

In [ ]:
users = pd.read_json("drive/MyDrive/yelp/Dataset Yelp/user.json", lines=True , chunksize=200000) #lee user.json
cods_users = pd.concat(users).user_id
cods_users = cods_users.reset_index()
cods_users.info()

In [ ]:
cods_users.index.duplicated().sum()
cods_users.user_id.duplicated().sum()

In [ ]:
cods_users = cods_users.rename(columns={'index':'n_user_id'})

In [ ]:
cods_users.to_csv('drive/MyDrive/yelp/codes_users', index=False)

In [ ]:
del cods_users

## user.json ETL

In [3]:
users = pd.read_json("drive/MyDrive/yelp/Dataset Yelp/user.json", lines=True , chunksize=400000) # definition of the iterator reader object

In [4]:
col_to_drop=['name','elite','compliment_hot','compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain',
'compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos'] # droppable columns list
data_types={'average_stars':'uint8' , 'fans':'uint16', 'review_count':'uint16', 'cool':'uint32', 'useful':'uint32', 'funny':'uint32'} # data types dict for some columns

chunks=[] # define empty list for chunks
for chunk in users:  #iterate over the reader object users
  iter_chunk = chunk.drop(columns=col_to_drop).astype(data_types) # drop columns listed in col_to_drop from every chunk and change the data types according to data_types dict
  chunks.append(iter_chunk)  # append modified chunks into the chunks list
  
  


In [5]:
users = pd.concat(chunks) # concatenate all the chunks in the chunks list to create users dataframe

In [6]:
users.yelping_since = pd.to_datetime(users.yelping_since, format='%Y-%m-%d %H:%M:%S') # change column 'yelping_since' to datetime

In [7]:
del chunks
del chunk
del iter_chunk

In [8]:
users = users.reset_index() # create a new column with integers from the index values, this new columns is named automatically as 'index'

In [9]:
users = users.rename(columns={'index':'n_user_id'}) # rename the recently created column of integers as n_user_id
users = users.astype({'n_user_id':'uint32'}) # change the type of column n_user_id to uint32

In [13]:
users.info()

Index(['n_user_id', 'user_id', 'review_count', 'yelping_since', 'useful',
       'funny', 'cool', 'friends', 'fans', 'average_stars'],
      dtype='object')

# Nueva sección